In [ ]:
import os
from langchain_openai import OpenAI

In [54]:
import os
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [3]:
from langchain_openai import AzureChatOpenAI

gpt = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo-1106",
    openai_api_key="7c3f9550b69c419aa0f1830e338ff562",
    openai_api_type="azure",
    openai_api_version="2023-12-01-preview",
    azure_endpoint="https://chatbotopenaikeyswe.openai.azure.com/",
    verbose=True,
    temperature=0.1,
)

In [4]:
from langchain_openai import AzureOpenAIEmbeddings
embed = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_key="7c3f9550b69c419aa0f1830e338ff562",
    azure_endpoint="https://chatbotopenaikeyswe.openai.azure.com/"
)

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import CSVLoader
path = "C:/Users/Admin/Desktop/hsag chatbot/CSVDATASET/Intent and Configuration Dataset"
csv_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="**/*.csv", loader_cls=CSVLoader, loader_kwargs=csv_loader_kwargs)
intents_configuration_data  = loader.load()

In [7]:
import requests

# Base URL of your mock server
base_url = "https://b07dcf0f-cb3c-49d3-9990-47521f1415d5.mock.pstmn.io"

# Customer data endpoint
meter_reading_url = f"{base_url}/meter/getMeterForCustomer"

# Meter reading data endpoint
customer_url = f"{base_url}/contract/getCustomerData"

# Send GET requests to each endpoint
customer_response = requests.get(customer_url)
meter_reading_response = requests.get(meter_reading_url)

# Handle responses
if customer_response.status_code == 200:
  customer_data = customer_response.json()
  print("Customer Data:", customer_data)
else:
  print("Error:", customer_response.status_code)

if meter_reading_response.status_code == 200:
  meter_reading_data = meter_reading_response.json()
  print("Meter Reading Data:", meter_reading_data)
else:
  print("Error:", meter_reading_response.status_code)


Customer Data: {'contracts': [{'contractNumber': 'ES00000062980', 'section': 'Gas', 'invoicingInterval': 12, 'cancellationDateTo': None, 'periodStart': '2022-03-14T23:00:00.000Z', 'periodEnd': None, 'state': 'GPKE_APPROVED', 'nextPossibleCancellationDate': '2024-03-14T22:59:59.000Z'}], 'firstName': 'Sebastian', 'lastName': 'Löecke', 'bankAccounts': [{'bankName': 'Commerzbank Köln', 'iban': 'DE89370400440532013000', 'bic': 'COBADEFFXXX'}], 'customerNumber': '1005520645', 'adress': {'street': 'Schmiedeweg', 'housenumber': '10b', 'postalCode': '21481', 'city': 'Lauenburg'}}
Meter Reading Data: {'meters': [{'meterId': 1244, 'meterNumber': 'ZEI2332DE2', 'section': 'Power', 'validFrom': None, 'validTo': '2023-05-04 23:59:59', 'readable': False, 'meterType': 'ZTZ', 'digitsBeforeDecimal': 5, 'digitsAfterDecimal': 0, 'register': [{'registerId': 1, 'obis': '1-1:1.8.1', 'label': 'HT', 'expectedLowerBound': 4554, 'expectedUpperBound': 5287}, {'registerId': 2, 'obis': '1-1:1.8.2', 'label': 'NT', 'e

In [8]:
erpdata = {**customer_data, **meter_reading_data}


In [10]:
path = "C:/Users/Admin/Desktop/hsag chatbot/CSVDATASET/Logdata"
csv_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="**/*.csv", loader_cls=CSVLoader, loader_kwargs=csv_loader_kwargs)
LogData_thuegaenergie = loader.load()

In [12]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.thuega-energie-gmbh.de/privatkunden.html"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
thuegaenergie_company_data = html2text.transform_documents(docs)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.98it/s]


In [14]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
intents_configuration_data_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [16]:
from langchain_text_splitters import RecursiveJsonSplitter
erpdata_text_splitter = RecursiveJsonSplitter(max_chunk_size=50)
#json_chunks = customer_data_text_splitter.split_json(json_data=customer_data)
chuncked_erp_data = erpdata_text_splitter.create_documents(texts=[erpdata])


In [17]:
LogData_thuegaenergie_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [18]:
thuegaenergie_company_data_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [19]:
chuncked_intents_configuration_data = intents_configuration_data_text_splitter.split_documents(intents_configuration_data)

In [20]:
chuncked_LogData_thuegaenergie = LogData_thuegaenergie_text_splitter.split_documents(LogData_thuegaenergie)

In [21]:
chuncked_thuegaenergie_company_data = thuegaenergie_company_data_text_splitter.split_documents(thuegaenergie_company_data)

In [22]:
from langchain_community.vectorstores import Chroma

In [ ]:
intents_configuration_data_vectorstore = Chroma.from_documents(chuncked_intents_configuration_data, embed, persist_directory="C:/chuncked_intents_configuration_db")
intents_configuration_data_vectorstore.persist()
erp_data_vectorstore = Chroma.from_documents(chuncked_erp_data, embed, persist_directory="C:/erp_db")
erp_data_vectorstore.persist()
LogData_thuegaenergie_vectorstore = Chroma.from_documents(chuncked_LogData_thuegaenergie, embed, persist_directory="C:/LogData_thuegaenergie_db")
LogData_thuegaenergie_vectorstore.persist()
thuegaenergie_company_data_vectorstore = Chroma.from_documents(chuncked_thuegaenergie_company_data, embed, persist_directory="C:/thuegaenergie_company_db")
thuegaenergie_company_data_vectorstore.persist()

In [23]:
intents_configuration_data_vectorstore2 = Chroma(persist_directory="C:/chuncked_intents_configuration_db", embedding_function=embed)
erp_data_vectorstore2 = Chroma(persist_directory="C:/erp_db", embedding_function=embed)
LogData_thuegaenergie_vectorstore2 = Chroma(persist_directory="C:/LogData_thuegaenergie_db", embedding_function=embed)
thuegaenergie_company_data_vectorstore2 = Chroma(persist_directory="C:/thuegaenergie_company_db", embedding_function=embed)


In [24]:
retriever1 = intents_configuration_data_vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k":3})
retriever2 = erp_data_vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k":3})
retriever4 = LogData_thuegaenergie_vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k":3})
retriever5 = thuegaenergie_company_data_vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [25]:
from langchain.tools.retriever import create_retriever_tool

In [26]:
intents_configuration_data_retriever_tool=create_retriever_tool(retriever1,"intents_configuration_search",
                      "Search for information about Intents and Configurations Overview. For any questions about Intents and Configurations, you must use this tool!")

erp_data_retriever_tool=create_retriever_tool(retriever2,"erpdatabase_search",
                      "Search for information about Customerdata and meterreading. For any questions about Customerdata and meterreading, you must use this tool!")

LogData_thuegaenergie_retriever_tool=create_retriever_tool(retriever4,"thuegaenergielogdata_search",
                      "Search for information about Logdata of Thuegaenergie. For any questions about Logdata of Thuegaenergie, you must use this tool!")

thuegaenergie_company_data_retriever_tool=create_retriever_tool(retriever5,"thuegaenergiecompany_search",
                      "Search for information about thuegaenergie company. For any questions about thuegaenergie company, you must use this tool!")


In [27]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [29]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [78]:
tools=[intents_configuration_data_retriever_tool, erp_data_retriever_tool, LogData_thuegaenergie_retriever_tool, thuegaenergie_company_data_retriever_tool, wiki ]

In [102]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [103]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(gpt,tools,prompt)

In [115]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)


In [117]:
agent_executor.invoke({"input":"How do I delete historical meter readings and what metering systems automate reading collection and consumption data?"})



> Entering new AgentExecutor chain...

Invoking: `erpdatabase_search` with `{'query': 'delete historical meter readings'}`


{"meters": [{"meterId": 1244, "meterNumber": "ZEI2332DE2", "section": "Power", "validFrom": null, "validTo": "2023-05-04 23:59:59", "readable": false, "meterType": "ZTZ", "digitsBeforeDecimal": 5, "digitsAfterDecimal": 0, "register": [{"registerId": 1, "obis": "1-1:1.8.1", "label": "HT", "expectedLowerBound": 4554, "expectedUpperBound": 5287}, {"registerId": 2, "obis": "1-1:1.8.2", "label": "NT", "expectedLowerBound": 3432, "expectedUpperBound": 3657}]}, {"meterId": 32423, "meterNumber": "972922", "section": "Power", "validFrom": "2023-05-05 00:00:00", "validTo": null, "readable": true, "meterType": "ZTZ", "register": [{"registerId": 1, "obis": "1-1:1.8.1", "label": "HT", "expectedLowerBound": 0, "expectedUpperBound": 658}, {"registerId": 2, "obis": "1-1:1.8.2", "label": "NT", "expectedLowerBound": 0, "expectedUpperBound": 360}]}]}

{"meters": [{"meterId": 1244

{'input': 'How do I delete historical meter readings and what metering systems automate reading collection and consumption data?',
 'output': 'To delete historical meter readings, you would need to contact your metering service provider or metering operator for assistance with this process. They will be able to guide you on the specific steps required to delete historical meter readings from your system.\n\nAs for metering systems that automate reading collection and consumption data, intelligent metering systems, also known as Smart Meters, are capable of automating the collection and storage of consumption data. These systems continuously register consumption and can transmit meter readings to the energy supplier without the need for manual reading. Modern metering devices can be upgraded with a gateway to enable remote reading, making them part of an intelligent metering system.\n\nIf you have further questions or need assistance with specific metering systems, I recommend reaching 

In [106]:
def get_response(question):
    result = agent_executor.invoke({"input": question})
    response = result.get("output", "")
    
    return {
        "answer": response,
    }
question = "What is the meter number for meter id 1244"
response = get_response(question)
print("Answer:", response["answer"])


Answer: The meter number for meter ID 1244 is "ZEI2332DE2".


Evaluation

In [107]:
from langsmith import Client
client = Client()

In [56]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith.evaluation import evaluate, LangChainStringEvaluator

In [108]:
inputs = [
    
    "What is the meter number for meterId 1244 and what is its meter type?",
    "What is the Gas contract number, start of the contract, city, of the customer named Sebastian?",
    "What services does Thuga Energie provide as a regional partner in the Hegau-Bodensee, Rhein-Pfalz, and Allgau-Oberschwaben regions?",
    "What options are there for making changes in our Kundenportal, and how can open questions be resolved with Kundenservice?",
    "What are PV Modules?",

    
]

outputs = [
    
    "The meter number for meterId 1244 is ZEI2332DE2 and its meter type is ZTZ",
    "The Gas contract number is ES00000062980, the IBAN is DE89370400440532013000, the start of the contract is 2022-03-14, and the city is Lauenburg",
    "Thuga Energie provides innovative products such as Erdgas, Strom, WÃrme, Photovoltaik, and Mobilität. They also offer comprehensive Beratung and Dienstleistungen to their customers.",
    "The Kundenportal offers the option to make changes online at any time, and open questions can be resolved with the Kundenservice through online communication. Additionally, human colleagues are available to assist with any queries. Rechnungen are received via email, and payments can be made through direct debit or bank transfer.",
    "PV modules are also known as photovoltaic modules, they are solar panels that convert sunlight into electricity.  The electricity generated by PV modules can be used for various purposes, including powering homes, businesses, and other electrical devices. PV modules are commonly used in solar energy systems to harness renewable energy from the sun.",
    
]
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]



In [109]:
client = Client()
dataset_name = "Agents_testset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about EnergyIndustry.",
)

In [110]:
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

In [60]:
from langchain_openai import AzureChatOpenAI

gpt4 = AzureChatOpenAI(
    azure_deployment="gpt-4-128k",
    openai_api_key="7c3f9550b69c419aa0f1830e338ff562",
    openai_api_type="azure",
    openai_api_version="2023-12-01-preview",
    azure_endpoint="https://chatbotopenaikeyswe.openai.azure.com/",
    verbose=True,
)

In [111]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response = get_response(example["question"])
    return {"answer": response["answer"]}

In [112]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_core.prompts.prompt import PromptTemplate
eval_llm = gpt4

qa_evaluator = [LangChainStringEvaluator("qa", config={"llm": eval_llm},
                                         prepare_data=lambda run, example: {
                                             "prediction": run.outputs["answer"],
                                             "reference": example.outputs["answer"],
                                             "input": example.inputs["question"],
                                         } )]

dataset_name = "Agents_testset"

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators= qa_evaluator,
    experiment_prefix="agents-qa-oai-qaevaluator",
    
    
)

View the evaluation results for experiment: 'agents-qa-oai-qaevaluator-bfaf34f9' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/da9f9c57-3cb1-496b-b04e-cac9093d895c/compare?selectedSessions=f6e523e1-2bdd-4d43-8b89-58da4c1e32d4




5it [00:11,  2.21s/it]


In [113]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_core.prompts.prompt import PromptTemplate
eval_llm = gpt4


cot_qa_evaluator = [LangChainStringEvaluator("cot_qa", config={"llm": eval_llm},
                                         prepare_data=lambda run, example: {
                                             "prediction": run.outputs["answer"],
                                             "reference": example.outputs["answer"],
                                             "input": example.inputs["question"],
                                         } )]
#qa_evalulator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "Agents_testset"

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators= cot_qa_evaluator,
    experiment_prefix="agents-qa-oai-cot-qaevaluator",
    
    
)

View the evaluation results for experiment: 'agents-qa-oai-cot-qaevaluator-15619381' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/da9f9c57-3cb1-496b-b04e-cac9093d895c/compare?selectedSessions=01e8e923-3ba8-40a8-813c-3979b311b9c9




5it [00:31,  6.33s/it]


In [114]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_core.prompts.prompt import PromptTemplate
eval_llm = gpt4


context_qa_evaluator = [LangChainStringEvaluator("context_qa", config={"llm": eval_llm},
                                         prepare_data=lambda run, example: {
                                             "prediction": run.outputs["answer"],
                                             "reference": example.outputs["answer"],
                                             "input": example.inputs["question"],
                                         } )]

dataset_name = "Agents_testset"

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators= context_qa_evaluator,
    experiment_prefix="agents-qa-oai-contextual accuracy",
    
    
)

View the evaluation results for experiment: 'agents-qa-oai-contextual accuracy-5c8819b2' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/da9f9c57-3cb1-496b-b04e-cac9093d895c/compare?selectedSessions=bc46789e-68f0-477a-9438-ee34a7ac3238




5it [00:10,  2.03s/it]
